## Extract image features

In [2]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18
from PIL import Image

# Load pre-trained ResNet model and remove the top fully connected layer
base_model = resnet18(pretrained=True)
base_model = torch.nn.Sequential(*(list(base_model.children())[:-1]))

batch = "batch1+2"

# Define image preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # the mean and std come from imagenet dataset
])

# Define dataset folder path
trainset = f'dataset/{batch}_train'
valset = f'dataset/{batch}_val'
testset = f'dataset/{batch}_test'

# Create an empty list to store features
train_features = []
val_features = []
test_features = []

# Iterate through all image files in the dataset folder
for filename in sorted(os.listdir(trainset)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Construct image path
        image_path = os.path.join(trainset, filename)

        # Load image
        img = Image.open(image_path)

        # Preprocess image and add batch dimension
        img_tensor = preprocess(img).unsqueeze(0)

        # Set model to evaluation mode
        base_model.eval()

        # Use ResNet model for feature extraction
        with torch.no_grad():
            features = base_model(img_tensor)

        # Append features to the list
        train_features.append(features.squeeze())


# Iterate through all image files in the dataset folder
for filename in sorted(os.listdir(valset)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Construct image path
        image_path = os.path.join(valset, filename)

        # Load image
        img = Image.open(image_path)

        # Preprocess image and add batch dimension
        img_tensor = preprocess(img).unsqueeze(0)

        # Set model to evaluation mode
        base_model.eval()

        # Use ResNet model for feature extraction
        with torch.no_grad():
            features = base_model(img_tensor)

        # Append features to the list
        val_features.append(features.squeeze())

# Iterate through all image files in the dataset folder
for filename in sorted(os.listdir(testset)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Construct image path
        image_path = os.path.join(testset, filename)

        # Load image
        img = Image.open(image_path)

        # Preprocess image and add batch dimension
        img_tensor = preprocess(img).unsqueeze(0)

        # Set model to evaluation mode
        base_model.eval()

        # Use ResNet model for feature extraction
        with torch.no_grad():
            features = base_model(img_tensor)

        # Append features to the list
        test_features.append(features.squeeze())


## Read labals 

In [3]:
import os
import pandas as pd

# Load label.csv file
label_df = pd.read_csv(f'label/{batch}label.csv')
labelname = "Firmness"

# Initialize lists to store labels
train_labels = []
val_labels = []
test_labels = []

# Iterate through all image files in the dataset folder
for filename in sorted(os.listdir(trainset)):
    if filename.endswith('.jpg'):
        # Extract filename without extension
        filename_without_extension = os.path.splitext(filename.replace("original_", ""))[0]

        # Extract label from label.csv based on filename
        label = label_df[label_df["Image_name"] == filename_without_extension][labelname].iloc[0]
        train_labels.append(label)

for filename in sorted(os.listdir(valset)):
    if filename.endswith('.jpg'):
        # Extract filename without extension
        filename_without_extension = os.path.splitext(filename)[0]
        
        # Extract label from label.csv based on filename
        label = label_df[label_df["Image_name"] == filename_without_extension][labelname].iloc[0]
        val_labels.append(label)

for filename in sorted(os.listdir(testset)):
    if filename.endswith('.jpg'):
        # Extract filename without extension
        filename_without_extension = os.path.splitext(filename)[0]
        
        # Extract label from label.csv based on filename
        label = label_df[label_df["Image_name"] == filename_without_extension][labelname].iloc[0]
        test_labels.append(label)

train_labels = [torch.tensor([float(train_labels[i])]) for i in range(len(train_labels))]
val_label = [torch.tensor([float(val_labels[i])]) for i in range(len(val_labels))]
test_label = [torch.tensor([float(test_labels[i])]) for i in range(len(test_labels))]

train_features_tensor = torch.stack(train_features)
train_labels_tensor = torch.tensor(train_labels)
val_features_tensor = torch.stack(val_features)
val_labels_tensor = torch.tensor(val_labels)
test_features_tensor = torch.stack(test_features)
test_labels_tensor = torch.tensor(test_labels)


## Random forest

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import numpy as np

# Define the random forest model
# 定义随机森林模型
model = RandomForestRegressor(random_state=42)

# Define the parameter grid for hyperparameter tuning
# 定义参数空间
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# Create a GridSearchCV object with parallel computation
# 创建 GridSearchCV 对象，并设置并行计算
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
# 拟合网格搜索模型
grid_search.fit(train_features_tensor, train_labels_tensor)

# Output the best parameter combination
# 输出最佳参数组合
print("Best Parameters:", grid_search.best_params_)

# Train the model with the best parameters
# 使用最佳参数训练模型
best_model = grid_search.best_estimator_
best_model.fit(train_features_tensor, train_labels_tensor)

# Make predictions on the training and validation sets
# 在训练集和验证集上进行预测
train_predictions = best_model.predict(train_features_tensor)
val_predictions = best_model.predict(val_features_tensor)

# Calculate mean squared error and R^2 score
# 计算均方误差和 R^2 分数
train_loss = mean_squared_error(train_labels_tensor, train_predictions)
val_loss = mean_squared_error(val_labels_tensor, val_predictions)
train_r2 = r2_score(train_labels_tensor, train_predictions)
val_r2 = r2_score(val_labels_tensor, val_predictions)

# Output performance metrics on the training and validation sets
# 输出训练和验证集上的性能指标
print(f'Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
print(f'Train R-squared: {train_r2:.4f}, Validation R-squared: {val_r2:.4f}')

# Make predictions on the test set
# 在测试集上进行预测
test_predictions = best_model.predict(test_features_tensor)
mse = mean_squared_error(test_labels_tensor, test_predictions)
r_squared = r2_score(test_labels_tensor, test_predictions)

# Output predictions and performance metrics on the test set
# 输出测试集上的预测结果和性能指标
print("Predictions:", test_predictions)
print("True Labels:", test_labels_tensor.tolist())
print(f'Test RMSE: {np.sqrt(mse):.4f}')
print(f'Test R-squared: {r_squared:.4f}')

Best Parameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Train Loss: 11.7437, Validation Loss: 77.7645
Train R-squared: 0.9730, Validation R-squared: 0.8213
Predictions: [22.60181714 34.3360951  34.14753005 30.5819763  39.8611784  47.00577013
 39.3172122  59.47892012 40.84183811 40.38413872 28.86888978 29.96007437
 21.12096804 25.62979906 27.56202948 26.09688982 31.0557143  57.37095736
 25.2629768  27.50913981 19.35330157 24.46892921 16.63358976 18.81722905
 16.60928397 16.05279959 21.08338882 24.69125074 20.43167192 15.36496859
 15.13977015 16.50971638 19.31800128 16.66770127 16.05681196 16.09728114
 13.89314644 14.70245332 27.2037913  17.03558088 14.54406089 14.26686662
  8.94098418 13.53704363 16.02327852 13.44702809 13.80097079 13.92125723
 14.47555403 13.12962855 14.35812389 14.14194235 14.76565634 14.43910135
 14.81501856 13.91956642 15.7972952  14.58787184 18.92000465 13.97311357
 15.18462898 15.14574048 12.97414486 12.95579172 12.87

## SVR

In [6]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import numpy as np

# Define the SVR model
# 定义 SVR 模型
svr = SVR()

# Define the parameter search space
# 定义参数搜索范围
param_grid = {
    'kernel': ['linear', 'poly', 'rbf'],  # Kernel types
    'C': [0.01, 0.1, 1, 10, 100],        # Regularization parameter
    'epsilon': [0.01, 0.1, 1, 10]        # Epsilon in the epsilon-SVR model
}

# Create the GridSearchCV object with parallel computation
# 创建 GridSearchCV 对象，并设置并行计算
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
# 拟合网格搜索模型
grid_search.fit(train_features_tensor, train_labels_tensor)

# Output the best parameter combination
# 输出最佳参数组合
print("Best Parameters:", grid_search.best_params_)

# Train the model with the best parameters
# 使用最佳参数训练模型
best_model = grid_search.best_estimator_
best_model.fit(train_features_tensor, train_labels_tensor)

# Make predictions on the training and validation sets
# 在训练集和验证集上进行预测
train_predictions = best_model.predict(train_features_tensor)
val_predictions = best_model.predict(val_features_tensor)

# Calculate mean squared error and R^2 score
# 计算均方误差和 R^2 分数
train_loss = mean_squared_error(train_labels_tensor, train_predictions)
val_loss = mean_squared_error(val_labels_tensor, val_predictions)
train_r2 = r2_score(train_labels_tensor, train_predictions)
val_r2 = r2_score(val_labels_tensor, val_predictions)

# Output performance metrics on the training and validation sets
# 输出训练和验证集上的性能指标
print(f'Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
print(f'Train R-squared: {train_r2:.4f}, Validation R-squared: {val_r2:.4f}')

# Make predictions on the test set
# 在测试集上进行预测
test_predictions = best_model.predict(test_features_tensor.numpy())
test_labels = test_labels_tensor.numpy()

# Calculate mean squared error and R^2 score on the test set
# 计算测试集上的均方误差和 R^2 分数
mse = mean_squared_error(test_labels, test_predictions)
r_squared = r2_score(test_labels, test_predictions)

# Output predictions and performance metrics on the test set
# 输出测试集上的预测结果和性能指标
print("Predictions:", test_predictions)
print("True Labels:", test_labels)
print(f'Test RMSE: {np.sqrt(mse):.4f}')
print(f'Test R-squared: {r_squared:.4f}')

Best Parameters: {'C': 100, 'epsilon': 0.01, 'kernel': 'poly'}
Train Loss: 9.5807, Validation Loss: 58.6827
Train R-squared: 0.9780, Validation R-squared: 0.8652
Predictions: [ 27.59938028  30.78437014  30.47175485  23.66307933  30.90298725
  38.21318946  29.03624389  58.71919304  35.28361465  34.54713098
  31.50670078  29.04101448  26.85074399  25.18764491  22.98983265
  16.03270666  32.65110321  34.05969653  21.41928798  31.2507285
  18.67750467  17.99437533  13.93499942  19.09881983  21.41977924
  15.13025737  20.71177489  12.09823407  15.76788654  17.51077578
  20.52236359  15.75274711  25.76317767  19.01925071  12.09121848
  17.05393944  16.93019385  17.67079717  19.03445277  12.86071699
  10.93074036  17.52701259   3.81101364   9.32418759  19.11695682
  13.90916561  10.97400934  14.75038896  12.17225043   6.86419847
   8.8501613   15.58834687  17.1080722   14.91194036  13.43476674
  15.75518977  17.8531072   15.16036385  13.6776388   16.33818698
   9.52118922  16.09028332  11.310